In [135]:
import logging
import mmap
import h5py
import torch
from torch.utils.data import Dataset
import numpy as np
import os
import pandas as pd
#import random

In [119]:
# read in the data
# data_folder = "/data/bvelghe/capstone2022/"
# data_set = "B/"
# file_path = "2018B.Sample.EOSlist.CTRL.p.v2.0.4_f.v2.0.4.h5"
# f = h5py.File(os.path.join(data_folder, data_set, file_path))

f = h5py.File("/data/bvelghe/capstone2022/C/2018E.EOSlist.CTRL_patched.h5")

In [123]:
def pion_hits_filt(file, start_ind, end_ind, p_lwr, p_upr):
    events_filt=[]
    n=0
    for i in range(start_ind, end_ind):

        hits_filtered=[]

        momentum=f['Events'][i][4]

        if p_lwr<=momentum<=p_upr:

            #saving the filtered event 
            events_filt.append(f['Events'][i])

            hits_data=f["Hits"][f["HitMapping"][i]:f["HitMapping"][i+1]]

            #obtaining the filtered hits
            for hit in hits_data:

                #checking if hit within specified range of chod time 
                if (f['Events'][i][5]-0.1)<=hit[4]<=(f['Events'][i][5]+0.1):
                    hits_filtered.append(hit)

            events_filt.append(np.array(hits_filtered))
            n+=1
    
        events_filt_np_pions=np.array(events_filt, dtype=object)
        
        
    return n, events_filt_np_pions

In [124]:
mu_off = f.attrs["muon_offset"]
pi_off = f.attrs["pion_offset"]
pos_off = f.attrs["positron_offset"]

In [7]:
def muon_sample(f,n_pions, p_lwr, p_upr):
    n=0
    j=0
    events_filt=[]
    
    #shuffling the index
    ind_mu=np.arange(mu_off, pi_off)
    muon_sample_id=np.random.choice(ind_mu, len(ind_mu))

    while n<n_pions:
        
        i=muon_sample_id[j]
        hits_filtered=[]
        momentum=f['Events'][i][4]
        j+=1


        if p_lwr<=momentum<=p_upr:

             #saving the filtered event 
            events_filt.append(f['Events'][i])

            hits_data=f["Hits"][f["HitMapping"][i]:f["HitMapping"][i+1]]

            #obtaining the filtered hits
            for hit in hits_data:
                #checking if hit within specified range of chod time 
                if (f['Events'][i][5]-0.1)<=hit[4]<=(f['Events'][i][5]+0.1):
                     hits_filtered.append(hit)

            events_filt.append(np.array(hits_filtered))
            n+=1

        events_filt_np_muons=np.array(events_filt, dtype=object)
        
        
    return n, events_filt_np_muons

In [ ]:
n_pions, pions_data=pion_hits_filt(f,pi_off,pi_off+5000,30,40)
muons_data=muon_sample(f,n_pions,30,40)
n_pions

In [ ]:
n_pions_20, pions_data_20=pion_hits_filt(f,pi_off,pos_off,20,30)
muons_data_20=muon_sample(f,n_pions,20,30)

In [133]:
def events_to_pandas(dfile):
    """Convert HDF5 events data to Pandas
    
    Parameters
    ----------
    dfile : HDF5 file
        The RICHAI HDF5 file to convert to pandas.
        
    Returns
    -------
    df : pd.DataFrame
        pandas DataFrame with Events data
    """
    
    df = pd.DataFrame()
    
    # event features
    df["run_id"] = dfile["Events"]["run_id"]
    df["burst_id"] = dfile["Events"]["burst_id"]
    df["event_id"] = dfile["Events"]["event_id"]
    df["track_id"] = dfile["Events"]["track_id"]
    df["track_momentum"] = dfile["Events"]["track_momentum"]
    df["chod_time"] = dfile["Events"]["chod_time"]
    df["ring_radius"] = dfile["Events"]["ring_radius"]
    df["ring_centre_pos_x"] = dfile["Events"]["ring_centre_pos"][:, 0]
    df["ring_centre_pos_y"] = dfile["Events"]["ring_centre_pos"][:, 1]
    df["ring_likelihood_pion"] = dfile["Events"]["ring_likelihood"][:, 0]
    df["ring_likelihood_muon"] = dfile["Events"]["ring_likelihood"][:, 1]
    df["ring_likelihood_positron"] = dfile["Events"]["ring_likelihood"][:, 2]
    
    # labels
    mu_off = dfile.attrs["muon_offset"]
    pi_off = dfile.attrs["pion_offset"]
    pos_off = dfile.attrs["positron_offset"]
    entries = dfile.attrs["entries"]
    
    labels = np.zeros(entries, dtype=np.int32)
    labels[mu_off:pi_off] = 0
    labels[pi_off:pos_off] = 1
    labels[pos_off:] = 2
    
    df["label"] = labels
    
    # hit mapping values
    df["first_hit"] = np.array(dfile["HitMapping"])[:-1]  # hit n
    df["last_hit"] = np.array(dfile["HitMapping"])[1:]    # hit n + 1
    df["total_hits"] = df["last_hit"] - df["first_hit"]
    
    return df


def get_class_samples(df, n, seed, momentum_bin=None, wide=None):
    """Sample n samples for each particle class from the events dataframe"""
    
    if momentum_bin and wide:
        df = df.query("wide_momentum_bin == @momentum_bin")
        
    if momentum_bin and not wide:
        df = df.query("narrow_momentum_bin == @momentum_bin")
        
    samples = pd.concat(
        [
            df[df["class"] == "muon"].sample(n=n, random_state=seed),
            df[df["class"] == "pion"].sample(n=n, random_state=seed),
            df[df["class"] == "positron"].sample(n=n, random_state=seed)        
        ]

    )
    
    return samples


In [171]:
def undersample(df):
    classes = df.label.value_counts().to_dict()
    least_class_amount = min(classes.values())
    classes_list = []
    for key in classes:
        classes_list.append(df[df['label'] == key]) 
    classes_sample = []
    for i in range(0,len(classes_list)-1):
        classes_sample.append(classes_list[i].sample(least_class_amount))
    df_maybe = pd.concat(classes_sample)
    final_df = pd.concat([df_maybe,classes_list[-1]], axis=0)
    final_df = final_df.reset_index(drop=True)
    return final_df

In [181]:
df=events_to_pandas(f)
df['original_index']=df.index
df=df.drop(columns=['first_hit', 'last_hit', 'total_hits'])
df=df[(df['label']==0) | (df['label']==1)]

In [182]:
df['momentum-bin']=np.where(
    (15<=df['track_momentum'])&(df['track_momentum']<25), '15-25', 
    np.where((25<=df['track_momentum'])&(df['track_momentum']<35), '25-35', 
         np.where((35<=df['track_momentum'])&(df['track_momentum']<45), '35-45', '>45')   
            )

)

In [183]:
df

,run_id,burst_id,event_id,track_id,track_momentum,chod_time,ring_radius,ring_centre_pos_x,ring_centre_pos_y,ring_likelihood_pion,ring_likelihood_muon,ring_likelihood_positron,label,original_index,momentum-bin
0,9040,873,26045,0,26.948496,19.262634,179.391769,-4.097735,31.231167,6.184088e-07,1.000000e+00,2.497073e-11,0,0,25-35
1,9040,873,60399,0,28.132404,5.083698,178.753250,-156.361954,-131.826675,5.162958e-06,1.000000e+00,2.491235e-11,0,1,25-35
2,9040,873,61482,0,31.447334,-7.501774,180.091827,-49.356911,-90.976852,2.885000e-07,1.000000e+00,1.508444e-07,0,2,25-35
3,9040,873,85696,0,17.581686,12.035756,160.696716,-54.584312,-29.460228,1.235752e-33,1.000000e+00,1.216099e-37,0,3,15-25
4,9040,873,163082,0,51.211544,8.200817,189.128403,-114.221886,57.083755,1.175196e-01,1.000000e+00,1.615611e-01,0,4,>45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9567645,8968,7,1232992,0,24.288748,10.672124,160.639084,-35.908329,28.821421,1.000000e+00,1.015724e-13,1.216099e-37,1,9567645,15-25
9567646,8968,7,1461058,0,36.989967,14.916398,180.464676,-186.481369,-47.899498,1.000000e+00,4.251668e-03,8.353033e-16,1,9567646,35-45
9567647,8968,7,1550399,0,14.598553,9.816546,102.037720,-159.958908,-77.076660,1.000000e+00,1.216099e-37,1.216099e-37,1,9567647,>45
9567648,8968,7,1617709,0,31.622683,9.481514,176.246384,-135.851852,-109.333832,1.000000e+00,1.447364e-02,4.835961e-10,1,9567648,25-35


In [184]:
bins=['15-25', '25-35', '35-45']

balanced_df={}
#sampling for 15<p<25

for i in bins: 
    balanced_df[i]=undersample(df[df['momentum-bin']==i])

In [240]:
path='/home/mukund/data'
#os.mkdir('/home/mukund/data')

for key, item in balanced_df.items():
    keys=key.split("-")
    save_path= f"{path}"+ "/datasetC_" + f"{keys[0]}_{keys[1]}" 
    item.to_hdf(save_path + ".h5",
                key=key,
                format="table",
                mode='w')

/home/mukund/.local/lib/python3.8/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: '15-25'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/mukund/.local/lib/python3.8/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: '25-35'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)
/home/mukund/.local/lib/python3.8/site-packages/tables/path.py:137: NaturalNameWarning: object name is not a valid Python identifier: '35-45'; it does not match the pattern ``^[a-zA-Z_][a-zA-Z0-9_]*$``; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  c

In [299]:
g=h5py.File('/fast_scratch_1/capstone_2022/datasetC_combined.h5')

In [294]:
g.keys()

<KeysViewHDF5 ['35-45']>

In [295]:
g['35-45']['table'][]

(10, [ 4.2286076e+01,  1.8607588e+01,  1.8730368e+02, -1.4463422e+02, -7.0665176e+01,  1.1758084e-07,  6.9643138e-03,  1.0000000e+00], [8972,  521,    0,    0], [ 182619, 7944022], [b'35-45'])

In [291]:
combined=pd.concat([balanced_df['15-25'], balanced_df['25-35'], balanced_df['35-45']], ignore_index=True)
combined.to_hdf('/home/mukund/data/datasetC_combined.h5',
                key='combined',
                format="table",
                mode='w')

In [263]:
combined.keys()

<KeysViewHDF5 ['combined']>

In [302]:
g['combined']['table'][1731225][4]

array([b'35-45'], dtype='|S5')

In [297]:
pd.concat([balanced_df['15-25'], balanced_df['25-35'], balanced_df['35-45']], ignore_index=True)

,run_id,burst_id,event_id,track_id,track_momentum,chod_time,ring_radius,ring_centre_pos_x,ring_centre_pos_y,ring_likelihood_pion,ring_likelihood_muon,ring_likelihood_positron,label,original_index,momentum-bin
0,9011,887,1800376,0,24.691673,16.837833,177.461960,-60.107861,106.794441,5.328789e-10,1.000000,9.393904e-21,0,2687658,15-25
1,8992,1059,770241,0,23.396029,9.284823,175.237564,-12.228768,11.319837,1.080940e-22,1.000000,6.127901e-07,0,4676657,15-25
2,9007,477,1922012,0,15.891424,22.654669,152.091827,-241.150375,-172.708389,1.216099e-37,1.000000,1.216099e-37,0,3262259,15-25
3,8987,1295,1634792,0,16.026428,23.254803,155.005783,-65.635712,-0.601753,1.216099e-37,1.000000,1.216099e-37,0,5473943,15-25
4,8971,654,2010374,0,21.888950,24.337191,172.810287,-231.750687,159.534821,4.307081e-14,1.000000,7.490480e-21,0,8071459,15-25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1731221,8968,7,563388,0,35.011139,16.212551,177.854492,-213.486633,-2.936464,1.000000e+00,0.000043,2.887881e-20,1,9567636,35-45
1731222,8968,7,572378,0,38.958267,19.714800,182.430420,-98.754082,92.138535,1.000000e+00,0.146916,3.528800e-07,1,9567637,35-45
1731223,8968,7,583135,0,38.474777,13.620609,176.691132,-145.546219,-77.260124,1.000000e+00,0.001099,5.301323e-11,1,9567638,35-45
1731224,8968,7,1461058,0,36.989967,14.916398,180.464676,-186.481369,-47.899498,1.000000e+00,0.004252,8.353033e-16,1,9567646,35-45


In [285]:
pd.concat([balanced_df['15-25'], balanced_df['25-35'], balanced_df['35-45']], ignore_index=True).columns

Index(['run_id', 'burst_id', 'event_id', 'track_id', 'track_momentum',
       'chod_time', 'ring_radius', 'ring_centre_pos_x', 'ring_centre_pos_y',
       'ring_likelihood_pion', 'ring_likelihood_muon',
       'ring_likelihood_positron', 'label', 'original_index', 'momentum-bin'],
      dtype='object')